<a href="https://colab.research.google.com/github/amirhoseinaghaei/Research_Simulation/blob/main/Research_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!fusermount -u drive
!google-drive-ocamlfuse drive

fusermount: failed to unmount /content/drive: No such file or directory
/bin/bash: google-drive-ocamlfuse: command not found


In [4]:
from google.colab import drive 
drive.mount('/content/gdrive/') 

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [5]:
%cd gdrive/MyDrive/Research_Simmulation/


/content/gdrive/MyDrive/Research_Simmulation


In [6]:



import numpy as np
# from Environment import Actor
from Rudder import LessonBuffer
from Environment import Environment
from Rudder import RRLSTM as LSTM
import torch
import time as Time
import random
from PolicyUpdater import PolicyUpdater

lb_size = 2048
n_lstm = 16
max_time = 50
policy_lr = 0.5
lstm_lr = 1e-2
l2_regularization = 1e-6
avg_window = 750

Lesson_buffer = LessonBuffer(1000, 25, 5)
episode = 0
rudder_lstm = LSTM(state_input_size=5, n_actions= 2, buffer=Lesson_buffer, n_units=n_lstm,
                        lstm_lr=lstm_lr, l2_regularization=l2_regularization, return_scaling=10,
                        lstm_batch_size=8, continuous_pred_factor=0.5)

# rudder_lstm.load_state_dict(torch.load('rudder_lstm.pt'))
environment = Environment(100,25)
environment.CreateStates()
# print(type(environment.StateList[0].Name))
policy_updator  = PolicyUpdater(environment= environment, lr = policy_lr)
episode = 0
visited_dict_0 = {}
visited_dict_1 = {}
for i in range(10000):
    episode += 1
    environment.reset_paramter()
    state = environment.reset_state()
    rewards = []
    states = [state]
    actions = []
    done = False
    name = f'({state[0]}, {state[1]}, {state[2]}, {state[3]}, {state[4]})'

    while not done:
        if episode <= 24:
          action = np.random.choice(2)
        else:
          if np.random.random() < 0.15:
              action = np.random.choice(2)
              if len(states) == 1 and action == 0:
                if name not in visited_dict_0.keys(): 
                    visited_dict_0[name] = 1
                else:
                    visited_dict_0[name] += 1
              if len(states) == 1 and action == 1:
                if name not in visited_dict_1.keys(): 
                    visited_dict_1[name] = 1
                else:
                    visited_dict_1[name] += 1  
      #    
          else:
            # if policy_updator.Quality[name,0] > policy_updator.Quality[name,1] else 1
              action = 0 if policy_updator.Quality[name,0] > policy_updator.Quality[name,1] else 1
              if len(states) == 1 and action == 0:
                if name not in visited_dict_0.keys(): 
                    visited_dict_0[name] = 1
                else:
                    visited_dict_0[name] += 1
              if len(states) == 1 and action == 1:
                if name not in visited_dict_1.keys(): 
                    visited_dict_1[name] = 1
                else:
                    visited_dict_1[name] += 1
        if environment.state.Ra == 0 and environment.state.U == 0:
            action = 0
        if environment.state.Ra == 0 and environment.state.U == 24:
            action = 1
        if environment.state.U > 0:
            action = 0
        state, reward, done = environment.step(action)
    
        actions.append(action)
        states.append(state)
        rewards.append(reward) 
        if done: 

            res = np.nonzero(rewards)[0]
            if len(res) > 0 :
              # print(res)
              rewards[-1] = rewards[res[0]]
              rewards[res[0]] = 0   
            for i in states: 
                if i[1] == "Ch1":
                    i[1] = 1
                else:
                    i[1] = 0
            states = np.stack(states)
            states = states.astype(int)
            rewards = np.array(rewards, dtype = np.float32)
            actions = np.array(actions)
            Lesson_buffer.add(states = states, actions = actions, rewards = rewards)
            if  episode < 2000 and Lesson_buffer.full_enough() and Lesson_buffer.different_returns_encountered()  :
                    # print("different_returns_encountered")        
                    # If RUDDER is run, the LSTM is trained after each episode until its loss is below a threshold.
                    # Samples will be drawn from the lessons buffer.
                    if episode % 25 == 0:

                        # print("True")
                        print(episode)
                        rudder_lstm.train(episode=episode)
                    if episode >= 1800: 
                        torch.save(rudder_lstm.state_dict(), 'rudder_lstm.pt')
                    # Then the LSTM is used to redistribute the reward.
            # print(rewards)
            # print(states)
            # print(actions)
            
            rewards = rudder_lstm.redistribute_reward(states=np.expand_dims(states, 0),actions=np.expand_dims(actions, 0))[0, :]
            policy_updator.Q_learning(actions= actions , states = states, rewards= rewards)
            # print(rewards)

for keys, value in policy_updator.Quality.items():
         initial_StateName = []
         for i in environment.initial_State:
            initial_StateName.append(i.Name) 
         if keys[0] in initial_StateName: 
            print('{:15} {:15} {:15}'.format( keys[0] ,  keys[1], value))
print("******************************************************************")
for keys, value in policy_updator.Quality.items():
         initial_StateName = []
         for i in environment.initial_State:
            initial_StateName.append(i.Name) 
         if keys[0] in initial_StateName: 
            if policy_updator.Quality[keys[0],0] > policy_updator.Quality[keys[0],1]:  
              print('{:15} {:15} {:15}'.format( keys[0] , "Wait" , policy_updator.Quality[keys[0] ,0]))
            else:
              print('{:15} {:15} {:15}'.format( keys[0] , "Send Back" , policy_updator.Quality[keys[0] ,1]))
print("******************************************************************")
for keys, value in policy_updator.Quality.items():
         initial_StateName = []
         for i in environment.initial_State:
            initial_StateName.append(i.Name) 
         if keys[0] in initial_StateName: 
            if policy_updator.Quality[keys[0],0] > 0:  
              print('{:15} {:15} {:15}'.format( keys[0] , "Wait" , policy_updator.Quality[keys[0] ,0]))
            else:
              print('{:15} {:15} {:15}'.format( keys[0] , "Send Back" , policy_updator.Quality[keys[0] ,1]))
print("******************************************************************")
for value in visited_dict_0:
        print('{:15} {:15} {:15}'.format("Wait", value, visited_dict_0[value]))


print("******************************************************************")
for value in visited_dict_1:
        print('{:15} {:15} {:15}'.format("Send Back", value, visited_dict_1[value]))


[[0.8, 0.19999999999999996], [0.81, 0.18999999999999995]]
[[0.8, 0.19999999999999996], [0.81, 0.18999999999999995]]
Amir
275
HI
BYE
300
HI
BYE
325
HI
BYE
350
HI
BYE
375
HI
BYE
400
HI
BYE
425
HI
BYE
450
HI
BYE
475
HI
BYE
500
HI
BYE
525
HI
BYE
550
HI
BYE
575
HI
BYE
600
HI
BYE
625
HI
BYE
650
HI
BYE
675
HI
BYE
700
HI
BYE
725
HI
BYE
750
HI
BYE
775
HI
BYE
800
HI
BYE
825
HI
BYE
850
HI
BYE
875
HI
BYE
900
HI
BYE
925
HI
BYE
950
HI
BYE
975
HI
BYE
1000
HI
BYE
1025
HI
BYE
1050
HI
BYE
1075
HI
BYE
1100
HI
BYE
1125
HI
BYE
1150
HI
BYE
1175
HI
BYE
1200
HI
BYE
1225
HI
BYE
1250
HI
BYE
1275
HI
BYE
1300
HI
BYE
1325
HI
BYE
1350
HI
BYE
1375
HI
BYE
1400
HI
BYE
1425
HI
BYE
1450
HI
BYE
1475
HI
BYE
1500
HI
BYE
1525
HI
BYE
1550
HI
BYE
1575
HI
BYE
1600
HI
BYE
1625
HI
BYE
1650
HI
BYE
1675
HI
BYE
1700
HI
BYE
1725
HI
BYE
1750
HI
BYE
1775
HI
BYE
1800
HI
BYE
1825
HI
BYE
1850
HI
BYE
1875
HI
BYE
1900
HI
BYE
1925
HI
BYE
1950
HI
BYE
1975
HI
BYE
(31, Ch1, 105, 1, 0)               0 -0.48231375217437744
(31, Ch1, 105, 1, 0)  